# Lab 01 - RDD vs DataFrame

Dalam latihan ini kita akan melihat perbedaan fungsi dan operasi antara RDD dan DataFrame, yaitu filtering dan agregasi.

Import package yang dibutuhkan

In [ ]:
import pyspark
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg

Create spark session

In [ ]:
spark = SparkSession.builder.appName('RDD & DataFrame').getOrCreate()

In [ ]:
spark.conf.get("spark.app.name")

## Create



In [ ]:
listData = [("Banten",1222258,"Lebak"),
("Banten",1175148,"Pandeglang"),
("Banten",1435003,"Serang"),
("Banten",2619803,"Tangerang"),
("DKI Jakarta",23340,"Kepulauan Seribu"),
("DKI Jakarta",2185711,"Jakarta Selatan"),
("DKI Jakarta",2843816,"Jakarta Timur"),
("DKI Jakarta",914182,"Jakarta Pusat"),
("DKI Jakarta",2463560,"Jakarta Barat"),
("DKI Jakarta",1747315,"Jakarta Utara")]

listData

### Create RDD

Create spark RDD dengan menggunakan fungsi `sparkContext.parallelize()`

In [ ]:
dataRDD = spark.sparkContext.parallelize(listData)

Tampilkan hasilnya dengan perintah `collect`

**Perhatikan** bahwa ketika kita memanggil perintah `collect()` terhadap sebuah RDD ataupun Dataset, **seluruh data dalam RDD tersebut akan dikirim ke node di mana driver berada**. Oleh karena itu perlu dipertimbangkan ukuran data sebelum memanggil fungsi `collect`.

In [ ]:
dataRDD.collect()

### Create DataFrame
Create DataFrame dengan menggunakan fungsi `createDataFrame()`

In [ ]:
dataDF = spark.createDataFrame(listData, ["province", "pop","district"])

Tampilkan hasilnya dengan perintah `show`

In [ ]:
dataDF.show()

### Filter data RDD

Untuk melakukan filtering RDD, kita menggunakan fungsi transformasi `filter`, dengan parameter berupa ekspresi `lambda`.

RDD tidak memiliki skema, sehingga kita mengakses baris dan kolom dengan menggunakan indeks.

Perhatikan bahwa perintah-perintah ini tidak langsung dijalankan oleh Spark, meskipun kita meng-execute cell yang bersangkutan.

In [ ]:
rddf = dataRDD.filter(lambda x: x[0] == 'Banten')

In [ ]:
rddf.collect()

### Filtering DataFrame

Untuk melakukan filtering, digunakan fungsi `filter`.

Karena DataFrame memiliki skema, maka kita dapat menggunakan nama kolom untuk melakukan filtering

In [ ]:
dff = dataDF.filter(dataDF['province'] == 'Banten')

Tampilkan hasilnya dengan perintah `show`

In [ ]:
dff.show()

## Group by dan Agregasi

### RDD

Untuk melakukan group by dan agregasi pada RDD, kita menggunakan fungsi transformasi `map` dan `reduceByKey`, dengan mengirimkan fungsi `lambda` sebagai parameter operasinya.

- `map` menerapkan ekspresi `lambda` pada **setiap element x** RDD
- `reduceByKey` melakukan operasi agregasi berdasar key tertentu sesuai ekspresi `lambda`

In [ ]:
# map 1       : tambahkan integer 1 di tiap row untuk melakukan count
# reduceByKey : jumlahkan (sum) kolom populasi dan count
# map 2       : hitung rata-rata dengan operasi : sum(populasi)/count

popRDD = (dataRDD
           .map(lambda x: (x[0], (x[1], 1)))
           .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
           .map(lambda x: (x[0], x[1][0]/x[1][1])))

Tampilkan hasilnya

In [ ]:
popRDD.collect()

### DataFrame

Groupby dan agregat dengan menggunakan fungsi `groupBy` dan `agg`.

In [ ]:
# Group by province name and average population
avgDF = dataDF.groupBy("province").agg(F.avg("pop"))

Tampilkan hasilnya

In [ ]:
# Show the results
avgDF.show()